In [19]:
## Read ParticleTable.txt

particles = {}

with open('data/ParticleTable.txt') as f:
    for line in f.readlines():
        if line.startswith('#') or line.startswith('PARTICLE') or line.startswith('END PARTICLE'): 
            continue
            
        els = line.split()
        name = els[0]
        pdgid = int(els[2])
        mass  = float(els[4])
        tlife = float(els[5])
        
        particles[pdgid] = {'name': name } #, 'mass': mass, 'tlife' : tlife
        
    

In [32]:
import ROOT as r

tf = r.TFile('data/p8_ee_Zbb_Bd2KstNuNu_stage1.root')
tree = tf.Get("events")

print_rows = [] 

nevents = 1000

for ev in range(nevents):
    tree.GetEntry(ev)
    for icand in range(int(tree.EVT_NKPi)):
        matchind = tree.KPiCandidates_truth[icand]
        
        # no truth match so can't do anything
        if matchind == -999: continue
        
        kpi_pdg = int(tree.MC_PDG[matchind])
        
        mother_inds = [tree.MC_M1[matchind], tree.MC_M2[matchind] ]
        
        # search for mothers and siblings
        sibling_inds = []
        mothers = []
        siblings = []
        for m_ind in mother_inds:
            if m_ind != -999:
                mothers.append( int(tree.MC_PDG[m_ind]) )
                for isib in range(1,5):
                    sib_ind = getattr(tree,f'MC_D{isib}')[m_ind]
                    if sib_ind != -999:
                        siblings.append( int(tree.MC_PDG[sib_ind]) )

        daughter_inds = [ tree.MC_D1[matchind], tree.MC_D2[matchind], 
                          tree.MC_D3[matchind], tree.MC_D4[matchind] ]
             
        daughters = []
        for d_ind in daughter_inds:
            if d_ind != -999:
                daughters.append( int(tree.MC_PDG[d_ind]) )
        
        kpi_name = particles[kpi_pdg]['name']
        
        mother_names = [ particles[m]['name'] for m in mothers ]
        if len(mother_names)==1: mother_names = mother_names[0]
        else: mother_names = ' '.join(mother_names)
            
        sibling_names = [ particles[s]['name'] for s in siblings]
        if len(sibling_names)==1: sibling_names = sibling_names[0]
        else: sibling_names = ' '.join(sibling_names)
            
        daughter_names = [ particles[d]['name'] for d in daughters ]
        if len(daughter_names)==1: daughter_names = daughter_names[0]
        else: daughter_names = ' '.join(daughter_names)
            
        print_rows.append( [ev, icand, kpi_name, mother_names, sibling_names, daughter_names])
        
        dec_descr = mother_names + ' -> ' + sibling_names + ' -> ' + daughter_names
        
        #print(ev, icand, dec_descr)
        
from tabulate import tabulate
print( tabulate(print_rows, headers=['Ev','iC','KstarCand','Mother','Siblings','Daughters']) )

  Ev    iC  KstarCand    Mother       Siblings                                                   Daughters
----  ----  -----------  -----------  ---------------------------------------------------------  -------------------
   0     0  K*(892)~0    B~0          K*(892)~0 nu_e~ nu_e                                       K- pi+
   1     0  K*(892)0     B0           K*(892)0 nu_e nu_e~                                        K+ pi-
   2     0  K*(892)0     B0           K*(892)0 nu_e nu_e~                                        K+ pi-
   2     1  D0           B-           D0 pi-                                                     K- pi+ pi0
   3     0  K*(892)~0    B~0          K*(892)~0 nu_e~ nu_e                                       K- pi+
   4     0  K*(892)0     B0           K*(892)0 nu_e nu_e~                                        K+ pi-
   5     0  K*(892)0     B0           K*(892)0 nu_e nu_e~                                        K+ pi-
   5     1  D~0          B+           D*_s+ 

In [ ]:
print(kpitruth)